In [1]:
import numpy as np
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure 
from bokeh.layouts import row
from ipywidgets import interact
output_notebook()

Loading BokehJS ...

In [60]:
# Initial values
dist_size = 150
car1_initpos = 2
car2_initpos = 0
car1_vel = 2
car2_initvel = 0
car1_acc = 0
car2_initacc = 0
dt = 0.01
dist_sep = 5

# Initializing arrays for plotting
timepoints = np.zeros(dist_size)
timepoints.fill(np.nan)
poscar1 = np.zeros(dist_size)
poscar1.fill(np.nan)
poscar1_est = np.zeros(dist_size)
poscar1_est.fill(np.nan)
poscar1_meas = np.zeros(dist_size)
poscar1_meas.fill(np.nan)
poscar2 = np.zeros([dist_size, 2])
poscar2.fill(np.nan)
car_sep = np.zeros([dist_size, 2])
car_sep.fill(np.nan)

In [64]:
# Setup plots
p1 = figure(plot_width=400, plot_height=300, x_axis_label='time', y_axis_label='position',
            x_range=[0, dist_size * dt], title='')
r1 = p1.line(timepoints, poscar1, line_color='cornflowerblue', legend='real car1',
            line_width=2)
r2 = p1.line(timepoints, poscar1_est, line_color='indianred', legend='predicted car1',
            line_width=2)
r3 = p1.line(timepoints, poscar2[:, 0], line_color='olivedrab', legend='car2 no kf',
            line_width=2)
r4 = p1.line(timepoints, poscar2[:, 1], line_color='plum', legend='car2 with kf',
            line_width=2)
p2 = figure(plot_width=400, plot_height=300, x_axis_label='time', y_axis_label='separation',
            x_range=[0, dist_size * dt], title='')
r5 = p2.line(timepoints, car_sep[:, 0], line_color='gold', line_width=2, legend='no KF')
r6 = p2.line(timepoints, car_sep[:, 1], line_color='darkslategray', line_width=2, legend='with KF')
p1.legend.location = 'bottom_right'  
p2.legend.location = 'bottom_center'

In [65]:
def RunCarFollowing():
    
    # Setup initial values
    pos1real = car1_initpos
    pos2 = car2_initpos
    pos2_kf = car2_initpos
    vel2 = car2_initvel
    vel2_kf = car2_initvel
    acc2 = car2_initacc
    acc2_kf = car2_initacc
    
    # Initialize Kalman Filter
    f = KalmanFilter(dim_x=2, dim_z=1)
    f.x = np.array([[car1_initpos],  # position
                    [car1_vel]])  # velocity
    f.F = np.array([[1.0, 1.0],
                    [0.0, 1.0]])
    f.H = np.array([[1.0, 0.0]])
    f.P = np.array([[1000.0, 0.0],
                    [0.0, 1000.0]])
    f.R = np.array([[5.0]])
    f.Q = Q_discrete_white_noise(dim=2, dt=dt, var=0.1)  # Process noise
    
    # Loop for updating positions
    for tt in range(dist_size):
        # Real position of leading car
        pos1real = pos1real + car1_vel * dt
        
        # Measurement of position of leading car with noise
        z = np.random.normal(pos1real, 1)
        
        # Kalman Filter predict and update steps
        f.predict()
        f.update(z)   
        
        # Following car update without Kalman Filter
        dx = z - pos2
        err = dx - dist_sep
        acc2 = (vel2**2 - car1_vel**2) / (2 * err)
        vel2 = vel2 + acc2 * dt
        pos2 = pos2 + vel2 * dt + 0.5 * acc2 * dt**2
        
        # Following car update with Kalman Filter
        dx_kf = f.x[0, 0] - pos2_kf
        err_kf = dx_kf - dist_sep
        acc2_kf = (vel2_kf**2 - car1_vel**2) / (2 * err_kf)
        vel2_kf = vel2_kf + acc2_kf * dt
        pos2_kf = pos2_kf + vel2_kf * dt + 0.5 * acc2_kf * dt**2
        
        # Update arrays for plotting
        global timepoints, poscar1, poscar2, poscar1_est, car_sep
        timepoints[tt] = tt * dt
        poscar1[tt] = pos1real
        poscar1_est[tt] = f.x[0, 0]
        poscar2[tt, 0] = pos2
        poscar2[tt, 1] = pos2_kf
        car_sep[tt, 0] = dx
        car_sep[tt, 1] = dx_kf   
        
        # Update plots
        r1.data_source.data['x'] = timepoints
        r1.data_source.data['y'] = poscar1
        r2.data_source.data['x'] = timepoints
        r2.data_source.data['y'] = poscar1_est
        r3.data_source.data['x'] = timepoints
        r3.data_source.data['y'] = poscar2[:, 0]
        r4.data_source.data['x'] = timepoints
        r4.data_source.data['y'] = poscar2[:, 1]
        r5.data_source.data['x'] = timepoints
        r5.data_source.data['y'] = car_sep[:, 0]
        r6.data_source.data['x'] = timepoints
        r6.data_source.data['y'] = car_sep[:, 1]
        push_notebook(handle=target)

In [66]:
target = show(row(p1, p2), notebook_handle=True)
interact(RunCarFollowing)

interactive(children=(Output(),), _dom_classes=('widget-interact',))